<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین سوم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش جست‌وجوی دارو</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
        <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>400200881</h5></td>
        <td style="border: none;"><h5>401204716</h5></td>
        <td style="border: none;"><h5>401204354</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> بهار ۱۴۰۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
    فایل ژوپیتر این تمرین در کولب توسعه داده و تست شده‌است. این فایل هم در محیط کولب، هم با ایمیج داکر jupyter/datascience-notebook تست شده‌است و همه قطعه‌کدها خروجی مورد انتظار را می‌دهند. اگه در بازتولید خروجی بعضی سل‌ها، یا کدهای تمرین مشکلی وجود داشت، ممنون می‌شویم در صورت امکان به ما اطلاع دهید تا فایل را در محیطی که قابل اجرا است، اجرا نموده و خروجی را نمایش دهیم. 
</b>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### واردکردن و نصب کتاب‌خانه‌های مورد استفاده


In [1]:
try:
    import ipywidgets
except:
    %pip install ipywidgets 
    
try: 
    import tensorflow as tf
except:
   %pip install tensorflow

try: 
    import keras_nlp
except: 
    %pip install keras-nlp

try:
    import torch
except:
    %pip install torch torchvision torchaudio
    
try: 
    import pandas as pd
except:
    %pip install pandas 

try: 
    import bert
except: 
    %pip install bert 
    
try: 
    import nltk
except: 
    %pip install nltk 
    

try:
    import gensim 
except:
    %pip install gensim 

try: 
    import langdetect
except: 
    %pip install langdetect 
    
try:
    import matplotlib as mpl
except:
    %pip install matplotlib 

In [2]:
# Some configuration for pandas for a better display of tables
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

In [3]:
from IPython.display import HTML

# Set the font family and fallback fonts for pandas output globally

mpl.rcParams['font.family'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.sans-serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']

def set_pandas_font(fonts):
    css = f"""
    <style>
        table.dataframe td, table.dataframe th {{
            font-family: {fonts};
        }}
    </style>
    """
    return HTML(css)

set_pandas_font("'vazirmatn', 'Vazir', 'B Nazanin', 'Arial'")

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **خلاصه‌ای از صورت مسئله و راه‌حل**
اینجا یه خلاصه می‌دیم که مسئله چیه. بعد یه outline از کارایی که کردیم می‌دیم. 

کارایی که کردیم (حالا به شکل بهتری بیان میشن):‌
۱- یافتن دیتابیسی از داروهای انگلیسی
۲- نرمالیزه کردن دیتابیس به شکلی که می‌خوایم 
۳- یافتن معادل فارسی داروهای دیتابیس 


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **مراحل طراحی سامانه جستجوی دارو**
در این بخش، مراحلی که برای طراحی سامانه جستجوی دارو طی شده‌اند و در قسمت پیشین خلاصه‌ای از آن بیان شد، به تفصیل شرح داده خواهد شد. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۱. یافتن دیتابیس داروها به زبان انگلیسی

در وب‌سایت سازمان غذا و داروی ایران، دیتاستی با حدود ۵۰ هزار ورودی که برای هر دارو شامل نام دارو به فارسی و لاتین، نام جنریک آن، نام برند و موارد دیگر می‌باشد، برای دانلود در دسترس عام است. دیتابیس اصلی استفاده شده در این تمرین، این دیتابیس می‌باشد. البته بر روی این دیتابیس، تغییرات و نرمال‌سازی‌هایی انجام شده‌است که در ادامه ذکر خواهند شد. 
دیتابیس دیگر، از این مخزن گیت‌هاب دانلود شد. این دیتابیس، شامل نام هر دارو به ازای کد ATC آن می‌باشد. کد ATC دارو، که در دیتاست سازمان غذا و داروی ایران نیز موجود است، طبقه‌بندی داروها رو توصیف می‌کند. برای مثال، برای داروی ملاتونین که یک قرص تنظیم ساعت خواب است، کد ATC آن N05CH01 می‌باشد. در این کد، کاراکترها به ترتیب، سلسله‌مراتب دسته‌ای که دارو به آن متعلق است را مشخص می‌کنند. برای داروی ذکرشده، این سلسله مراتب به شرح زیر است:
- N: سیستم عصبی
- N05: داروهای روان‌پزشکی
- N05C: خواب‌آور و آرام‌بخش
- N05CH: آگونیست گیرنده‌های ملاتونین
- N05CH01: قرص ملاتونین ۲ میلی‌گرم

همان‌طور که از این مثال پیداست، کد ATC دارو اطلاعات ارزشمندی در خصوص دسته‌بندی، و کارکرد آن به ما می‌دهد. با توجه به اینکه دیتاست اصلی سازمان غذا و دارو، برای هر دارو کد ATC آن را نیز دارد، با Join نمودن این دو دیتاست، توانستیم برای هر دارو، دسته‌بندی و کاربردهایی از آن را در قالب چند ستون اضافه، به دیتاست اضافه نماییم. 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۲. نرمال‌سازی و پیش‌پردازش‌های روی دیتابیس

### حذف داده‌های اضافه
دیتابیس دارهای سازمان غذا و دارو، دارای ستون‌هایی بود که برای این تمرین، کاربردی نداشتند. این ستون‌ها، شامل ستون‌هایی نظیر آمار فروش و قیمت داروها، و امکان فروش بدون نسخه داروها بودند. از آن‌جایی که این‌کار پیچیدگی خاصی نداشت، ستون‌های اضافه به صورت دستی از فایل حذف شدند. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### نرمال‌سازی داده‌ها
برخی نرمال‌سازی‌ها بر روی دیتاست داروها انجام گردید. مقدار تمام مدخل‌های جدول، به فرم lowercase آن تبدیل شد.  

In [4]:
IR_FDA_DATASET_PATH = "./data/Iran_FDA_1400_Dataset.csv"
WHO_ATC_INDEX_PATH = "./data/WHO_ATC_Index.csv"

medicine_ds = pd.read_csv(IR_FDA_DATASET_PATH)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,Actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR,J05AX
1,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1mL 20 mg,"REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mg","REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mL",REMDESIVIR,J05
2,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"NOLPAZA TABLET, DELAYED RELEASE ORAL 40 mg","PANTOPRAZOLE TABLET, DELAYED RELEASE ORAL 40 mg",PANTOPRAZOLE,A02BC02
3,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,ASENTRA TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE) TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE),N06AB06
4,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1mL 0.4 mL,NOXPRIN INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM,B01AB05


In [5]:
atc_index_ds = pd.read_csv(WHO_ATC_INDEX_PATH)

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,A,ALIMENTARY TRACT AND METABOLISM,NaN,NaN,NaN,NaN
1,A01,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
2,A01A,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
3,A01AA,Caries prophylactic agents,NaN,NaN,NaN,NaN
4,A01AA01,sodium fluoride,1.1,mg,O,0.5 mg fluoride


In [6]:
import pandas as pd
from langdetect import detect

# Lowercase all entries in the drug dataset
medicine_ds[medicine_ds.select_dtypes(['object']).columns] = medicine_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b01ab05


In [7]:
# Lowercase all entries in the ATC index dataset
atc_index_ds[atc_index_ds.select_dtypes(['object']).columns] = atc_index_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,a,alimentary tract and metabolism,NaN,NaN,NaN,NaN
1,a01,stomatological preparations,NaN,NaN,NaN,NaN
2,a01a,stomatological preparations,NaN,NaN,NaN,NaN
3,a01aa,caries prophylactic agents,NaN,NaN,NaN,NaN
4,a01aa01,sodium fluoride,1.1,mg,o,0.5 mg fluoride


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### جداسازی تکه‌های معنادار کد ATC داروها
همان‌طور که در بالا گفته شد، کد ATC داروها شامل قسمت‌های معناداری است که به صورت سلسله‌مراتبی، طبقه‌بندی دارو،‌و حوزه استفاده آن را مشخص می‌کند. در این قسمت، کد ATC هر دارو، مطابق مستنداتی که ساختار کد ATC را شرح داده‌اند، به ۵ قسمت تقسیم شده و این ۵ قسمت، در قالب ۵ ستون (ویژگی) به دیتاست اصلی اضافه شدند. در زیر، کد مربوط به تقسیم شناسه ATC داروها پیاده‌سازی شده‌است. 


In [8]:
def extract_atc_parts(atc_code):
    atc_code = str(atc_code)
    return [atc_code[:1], atc_code[:3] if len(atc_code) >= 3 else None,
            atc_code[:4] if len(atc_code) >= 4 else None, atc_code[:5] if len(atc_code) >= 5 else None,
            atc_code if len(atc_code) >= 7 else None]
 
ATC_CATEGORIES = ['anatomical', 'therapeutic', 'pharmacological', 'subpharmacological', 'chemical']
medicine_atc_splitted = medicine_ds['atc_code'].apply(extract_atc_parts).to_list()

medicine_atc_columns = pd.DataFrame(medicine_atc_splitted, dtype='object', columns=ATC_CATEGORIES)

medicine_ds = medicine_ds.join(medicine_atc_columns, how='inner')
medicine_ds.drop(['atc_code'], axis=1, inplace=True)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,anatomical,therapeutic,pharmacological,subpharmacological,chemical
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j,j05,j05a,j05ax,None
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j,j05,None,None,None
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a,a02,a02b,a02bc,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n,n06,n06a,n06ab,n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b,b01,b01a,b01ab,b01ab05


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۳. ترکیب دیتاست داروها با دیتاست طبقه‌بندی کد ATC داروها
اینجا توضیح میدیم دو تا دیتاست چجوری جوین شدن

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۴. جداسازی دیتابیس‌های فارسی و انگلیسی
توضیحات در خصوص تقسیم دیتابیس به دو تا دیتابیس فارسی و انگلیسی

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## **ارزیابی عملکرد سامانه**

